In [2]:
from rdkit.Chem import AllChem
from scipy import stats
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns
import time
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [3]:
import pandas as pd
import numpy as np

In [4]:
pubchem_data = pd.read_csv("./Pubchem_92_assays_binary.csv") 
assay_list= pubchem_data.columns.to_list()[1:93]
assay_data= pubchem_data.iloc[:, :93]
assay_data.to_csv("Pubchem_data_filtered_92_inchi.csv", index=False)

In [4]:
CP_features = pd.read_csv("./CP_filtered_10402.csv") 
CP_features_list= CP_features.columns.to_list()[:-1]
#CP_features = CP_features.to_numpy()
CP_features

,Cells_Correlation_Correlation_DNA_AGP,Cells_Correlation_Correlation_ER_AGP,Cells_Correlation_K_AGP_DNA,Cells_Correlation_K_DNA_AGP,Cells_Correlation_K_DNA_ER,Cells_Correlation_K_DNA_Mito,Cells_Correlation_K_ER_DNA,Cells_Correlation_K_ER_Mito,Cells_Correlation_K_Mito_DNA,Cells_Correlation_K_RNA_DNA,...,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_SumVariance_RNA_5_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,StdInChI
0,-0.045399,-0.052834,-0.006567,0.011172,0.009235,-0.013632,-0.012062,-0.012975,-0.010769,-0.016556,...,-0.042175,-0.126538,-0.135921,-0.218352,-0.348235,0.022734,-0.064442,-0.049853,-0.037229,InChI=1S/C11H14ClN3O4S3/c1-2-3-20-6-11-14-8-4-...
1,0.129807,0.068373,-0.033614,0.071465,0.076745,0.156491,-0.040814,0.077470,-0.102940,-0.031626,...,-0.174593,-0.174820,0.067516,0.017844,-0.137245,0.006468,-0.029972,0.017667,0.070510,InChI=1S/C25H27ClN2O/c26-22-12-7-13-23(18-22)2...
2,0.021885,0.023282,0.035352,-0.056394,-0.055012,-0.060501,0.024850,-0.000387,0.045720,0.039360,...,0.298685,-0.034911,-0.311239,-0.173322,-0.328665,0.031219,0.020276,-0.050662,-0.056293,InChI=1S/C18H20O2/c1-18-9-8-14-13-5-3-12(19)10...
3,0.047799,0.027258,-0.000531,-0.029938,-0.047386,-0.004365,0.010389,0.017189,-0.008475,0.063592,...,-0.219672,-0.183920,-0.057310,-0.032487,-0.227981,-0.115135,-0.108891,-0.022526,-0.030227,InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(1...
4,0.026384,0.012069,-0.038810,0.064665,0.072837,0.039043,-0.038961,-0.011498,-0.063074,-0.049690,...,-0.106546,-0.114580,-0.331848,-0.072634,-0.193127,-0.059346,0.013632,-0.157595,0.010668,InChI=1S/C10H14ClN3/c11-9-2-1-3-10(13-9)14-6-4...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10397,-0.009067,0.028331,-0.073881,0.061545,0.072425,0.080479,-0.029669,0.038259,-0.087029,-0.124156,...,0.172473,0.109905,-0.281850,-0.086699,-0.249878,0.028272,0.077785,-0.046787,-0.064416,"InChI=1S/C34H41N5O3S/c1-34(2,3)43(42)39-23-28-..."
10398,-0.020739,-0.017560,-0.022186,-0.004646,-0.003457,0.042389,0.005012,0.052370,-0.063380,-0.014991,...,-0.064359,-0.059644,-0.276168,-0.168500,-0.332086,-0.020784,-0.040793,-0.123188,0.009448,"InChI=1S/C34H36N4O3S/c1-34(2,3)42(41)38-22-28-..."
10399,-0.002143,0.009537,0.002578,-0.008484,-0.022082,0.008640,-0.002887,0.025904,-0.015267,-0.013379,...,-0.255099,-0.049035,-0.111082,-0.148793,-0.180716,0.013233,0.003183,-0.033117,-0.020764,"InChI=1S/C33H39N5O3S/c1-33(2,3)42(41)38-22-27-..."
10400,0.351428,0.342367,0.239651,-0.065820,-0.026131,0.124606,0.039575,0.142086,-0.061881,0.139468,...,-0.168908,0.570236,-0.251480,-0.552330,-0.725024,-0.234750,0.463339,0.091231,-0.263527,InChI=1S/C27H26ClFN2O6S/c28-17-3-1-16(2-4-17)1...


In [5]:
mfp_features = pd.read_csv("./Mfp_filtered_10402.csv") 
mfp_features_list = mfp_features.columns.to_list()[:-1]
mfp_features['fp'] = [(x) for x in mfp_features.iloc[:, :-1].to_numpy()]
mfp_features = mfp_features[["StdInChI", "fp"]]
mfp_features

,StdInChI,fp
0,InChI=1S/C11H14ClN3O4S3/c1-2-3-20-6-11-14-8-4-...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C25H27ClN2O/c26-22-12-7-13-23(18-22)2...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
2,InChI=1S/C18H20O2/c1-18-9-8-14-13-5-3-12(19)10...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(1...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,InChI=1S/C10H14ClN3/c11-9-2-1-3-10(13-9)14-6-4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
10397,"InChI=1S/C34H41N5O3S/c1-34(2,3)43(42)39-23-28-...","[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
10398,"InChI=1S/C34H36N4O3S/c1-34(2,3)42(41)38-22-28-...","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
10399,"InChI=1S/C33H39N5O3S/c1-33(2,3)42(41)38-22-27-...","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
10400,InChI=1S/C27H26ClFN2O6S/c28-17-3-1-16(2-4-17)1...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, ..."


In [6]:
pubchem_data= pd.merge(assay_data, CP_features)
pubchem_data= pd.merge(pubchem_data, mfp_features)
pubchem_data

,StdInChI,588458,588334,2642,2156,2330,2216,743015,504444,894,...,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_SumVariance_RNA_5_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,fp
0,InChI=1S/C11H14ClN3O4S3/c1-2-3-20-6-11-14-8-4-...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,...,-0.042175,-0.126538,-0.135921,-0.218352,-0.348235,0.022734,-0.064442,-0.049853,-0.037229,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C25H27ClN2O/c26-22-12-7-13-23(18-22)2...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,-0.174593,-0.174820,0.067516,0.017844,-0.137245,0.006468,-0.029972,0.017667,0.070510,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
2,InChI=1S/C18H20O2/c1-18-9-8-14-13-5-3-12(19)10...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,...,0.298685,-0.034911,-0.311239,-0.173322,-0.328665,0.031219,0.020276,-0.050662,-0.056293,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C16H13ClN2O/c1-19-14-8-7-12(17)9-13(1...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,-0.219672,-0.183920,-0.057310,-0.032487,-0.227981,-0.115135,-0.108891,-0.022526,-0.030227,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,InChI=1S/C10H14ClN3/c11-9-2-1-3-10(13-9)14-6-4...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,...,-0.106546,-0.114580,-0.331848,-0.072634,-0.193127,-0.059346,0.013632,-0.157595,0.010668,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10397,"InChI=1S/C34H41N5O3S/c1-34(2,3)43(42)39-23-28-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.172473,0.109905,-0.281850,-0.086699,-0.249878,0.028272,0.077785,-0.046787,-0.064416,"[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
10398,"InChI=1S/C34H36N4O3S/c1-34(2,3)42(41)38-22-28-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.064359,-0.059644,-0.276168,-0.168500,-0.332086,-0.020784,-0.040793,-0.123188,0.009448,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
10399,"InChI=1S/C33H39N5O3S/c1-33(2,3)42(41)38-22-27-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.255099,-0.049035,-0.111082,-0.148793,-0.180716,0.013233,0.003183,-0.033117,-0.020764,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
10400,InChI=1S/C27H26ClFN2O6S/c28-17-3-1-16(2-4-17)1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.168908,0.570236,-0.251480,-0.552330,-0.725024,-0.234750,0.463339,0.091231,-0.263527,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, ..."


In [7]:
def TanimotoSimilarity(ex1, ex2, attrs):
  """
    >>> v1 = [0,1,0,1]
    >>> v2 = [1,0,1,0]
    >>> TanimotoDist(v1,v2,range(4))
    1.0
    """
  inter = 0.0
  unin = 0.0
  for i in attrs:
    if (ex1[i] or ex2[i]):
      unin += 1
      if (ex1[i] and ex2[i]):
        inter += 1
  if (unin != 0.0):
    return (inter / unin)
  else:
    return 0

In [8]:
def BulkTanimoto(siv1, sivs):
    res = []
    for i in range(len(sivs)): 
        siv2 = sivs[i]
        simVal = TanimotoSimilarity(siv1, siv2, range(len(siv1)))
        res.append(simVal)
    return res

In [9]:
#Find similarity between each test compounds and train set compounds of same class and 
#store the lowest tanimoto similarity of MFP keys
neighbours_limit = 5
def findsimscore(assay, train, test): 
    ts_nearest_list = []

    for index, row in test.iterrows():

        fps_test = row["fp"]

        compound_activity= row[assay]
        #print(compound_activity)
        temp_train = pd.DataFrame()
        temp_train= train[train[assay] == compound_activity]
        #print(temp.shape)

        fps_temp_train=[(x) for x in temp_train["fp"]]
        ts_nearest = []
        #  median of 5 most similar compounds by sorted assay in descending order
        ts_nearest = (np.sort(BulkTanimoto(fps_test ,fps_temp_train))[::-1])[:neighbours_limit]
        ts_nearest=np.median(ts_nearest)
        ts_nearest_list.append(ts_nearest)

    ts_nearest_list=np.array(ts_nearest_list)
    ts_nearest_list= ts_nearest_list.reshape(len(test))

    df_ts_nearest = pd.DataFrame({ "ts_nearest": ts_nearest_list})
    return df_ts_nearest

In [10]:
#Find similarity between each test compounds and train set compounds of same class and 
#
neighbours_limit = 5
def findpearsonscore(assay, train, test): 
    pc_highest_list = []

    for index, row in test.iterrows():

        fps_test = row[CP_features_list].to_numpy()

        compound_activity= row[assay]
        #print(compound_activity)
        temp_train = pd.DataFrame()
        temp_train= train[train[assay] == compound_activity]
        #print(temp.shape)

        fps_temp_train=[(x) for x in temp_train[CP_features_list].to_numpy()]
        pearson_s_list = []
        
        for n in range(len(fps_temp_train)):
            
            pearson= stats.pearsonr(fps_test, fps_temp_train[n])
            pearson_s_list.append(pearson[0])
            #median of 15 positive correlations
            
        pc_highest= np.sort(pearson_s_list)[len(fps_temp_train)-neighbours_limit:]
        pc_lowest= np.sort(pearson_s_list)[:neighbours_limit]    
        
        pc_highest=(np.median(pc_highest))#+ abs(np.median(pc_lowest))) /2
        pc_highest_list.append(pc_highest)
    
    pc_highest_list=np.array(pc_highest_list)
    pc_highest_list= pc_highest_list.reshape(len(test))

    df_pc_highest = pd.DataFrame({ "pc_highest": pc_highest_list})
    return df_pc_highest

In [11]:
def plot_hist_ts(df):
    
    #f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
    
    mean=df['ts_nearest'].mean()
    median=df['ts_nearest'].median()
    mode=df['ts_nearest'].mode().tolist()[0]
    
    
    #sns.boxplot(x=df["td_nearest"], ax=ax_box)
    #ax_box.axvline(mean, color='r', linestyle='--',  label='Mean')
    #ax_box.axvline(median, color='g', linestyle='-',  label='Median')
    #ax_box.axvline(mode, color='b', linestyle='-',  label='Mode')

    #sns.histplot(x=df["td_nearest"],  kde=True, ax=ax_hist)
    #ax_hist.axvline(mean, color='r', linestyle='--', label='Mean')
    #ax_hist.axvline(median, color='g', linestyle='-', label='Median')
    #ax_hist.axvline(mode, color='b', linestyle='-', label='Mode')

    #plt.legend()
    #ax_box.set(xlabel='')
    #plt.show()
    
    return mean, median, mode

def plot_hist_pc(df):
    
    #f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
    
    mean=df['pc_highest'].mean()
    median=df['pc_highest'].median()
    mode=df['pc_highest'].mode().tolist()[0]
    
    
    #sns.boxplot(x=df["pc_highest"], ax=ax_box)
    #ax_box.axvline(mean, color='r', linestyle='--',  label='Mean')
    #ax_box.axvline(median, color='g', linestyle='-',  label='Median')
    #ax_box.axvline(mode, color='b', linestyle='-',  label='Mode')

    #sns.histplot(x=df["pc_highest"],  kde=True, ax=ax_hist)
    #ax_hist.axvline(mean, color='r', linestyle='--', label='Mean')
    #ax_hist.axvline(median, color='g', linestyle='-', label='Median')
    #ax_hist.axvline(mode, color='b', linestyle='-', label='Mode')

    #plt.legend()
    #ax_box.set(xlabel='')
    #plt.show()
    
    return mean, median, mode

In [12]:
#Nested Cross validation
#stat_list_batch: contains assay number, mean, median, mode of tanimoto distance in that cycle, mcc and auc of that cycle
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts
#pred_list_batch contains proba, pred and true values

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler 

from collections import Counter
from numpy import argmax

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from scipy.stats import ks_2samp 
from scipy.stats import randint
from sklearn.model_selection import cross_val_predict

def my_function_nested_rhs(assay, cycle):   
        
    detail_list_batch_train = []

    X_fp = df_assay_train.loc[:, "fp"].to_numpy()
    X_CP = df_assay_train[CP_features_list].to_numpy()
    
    #print(X)
    y = df_assay_train[assay].to_numpy()
    #print(y)
    
    
    #Split
    outercv = StratifiedKFold(n_splits=5,shuffle=True, random_state=cycle)
   
    for i, (train_index, test_index) in enumerate(outercv.split(X_fp, y)):
        
        train= df_assay_train.iloc[train_index]
        test= df_assay_train.iloc[test_index]
        #print(train)
        #print(test)

        df_ts_highest = findsimscore(assay, train, test)
        df_pc_highest = findpearsonscore(assay, train, test)    
        
        mean_ts, median_score, mode_score= plot_hist_ts(df_ts_highest)
        mean_pc, median_score, mode_score= plot_hist_pc(df_pc_highest)
        
        
        #Prepare features
        train_x_fp, test_x_fp, train_y, test_y = list(train.fp), list(test.fp), train[assay], test[assay]
        col_list=[]
      
        train_x_cp=train[CP_features_list].to_numpy()
        test_x_cp=test[CP_features_list].to_numpy()
        
        #MFP
        print("Mfp_Model_Running for assay outercv#", i+1)    
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
         
        param_dist_grid = {
        'max_depth': [10,15,20],
        #'max_features': randint(40, 50),
        'min_samples_leaf': randint(2, 15),
        'min_samples_split': randint(2, 15),
        'n_estimators':[100, 300, 400, 500],
        'bootstrap': [True, False],
        'oob_score': [False],
        'random_state': [42],
        'criterion': ['gini', 'entropy'],
        'n_jobs': [-1],
        'class_weight' : [None, 'balanced']
        }
        
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)

        rsh.fit(train_x_fp, train_y)

        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)                      
        classifier.fit(train_x_fp, train_y)
        
        #Threshold Balancing
        cross_val_prob_fp = cross_val_predict(rsh.best_estimator_, train_x_fp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_fp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_fp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_fp))


        fp_proba = classifier.predict_proba(test_x_fp)[:,1]
        #fp_pred = classifier.predict(test_x_fp) 
        fp_pred  = [ 1 if x>best_thresh_fp  else 0 for x in fp_proba ] 
    
        #Cell Painting
        
        print("CP_Model_Running for assay outercv#", i+1)  
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)
        
        
        rsh.fit(train_x_cp, train_y)
        
        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_    
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)
        classifier.fit(train_x_cp, train_y)
        
        #Threshold Balancing
        cross_val_prob_cp = cross_val_predict(rsh.best_estimator_, train_x_cp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_cp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_cp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_cp))


        CP_proba = classifier.predict_proba(test_x_cp)[:,1]
        #CP_pred = classifier.predict(test_x_cp) 
        CP_pred  = [ 1 if x>best_thresh_cp  else 0 for x in CP_proba ] 
        
        fpr, tpr, _ = roc_curve(test_y, CP_proba)

        #combine results
        for StdInChI, fp_proba, fp_pred, CP_proba, CP_pred, true_val, ts, pc in zip(test.StdInChI, fp_proba, fp_pred, CP_proba, CP_pred, test_y, df_ts_highest.ts_nearest,  df_pc_highest.pc_highest):
            detail_list_batch_train.append([assay, StdInChI, fp_proba, fp_pred,best_thresh_fp, CP_proba, CP_pred, best_thresh_cp, true_val, ts, pc, "Training"])
    
    return detail_list_batch_train

In [13]:
#Predict External Test Set
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts


def my_function_heldout_rhs(assay, cycle):   
        
        detail_list_batch_test = []

        train= df_assay_train
        test= df_assay_test

        df_ts_highest = findsimscore(assay, train, test)
        df_pc_highest = findpearsonscore(assay, train, test)    
        
        mean_ts, median_score, mode_score= plot_hist_ts(df_ts_highest)
        mean_pc, median_score, mode_score= plot_hist_pc(df_pc_highest)
        
        
        #Prepare features
        train_x_fp, test_x_fp, train_y, test_y = list(train.fp), list(test.fp), train[assay], test[assay]
        col_list=[]
      
        train_x_cp=train[CP_features_list].to_numpy()
        test_x_cp=test[CP_features_list].to_numpy()
        
        #MFP
        print("Mfp_Model_Running for assay")    
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
         
        param_dist_grid = {
        'max_depth': [10,15,20],
        #'max_features': randint(40, 50),
        'min_samples_leaf': randint(2, 15),
        'min_samples_split': randint(2, 15),
        'n_estimators':[100, 300, 400, 500],
        'bootstrap': [True, False],
        'oob_score': [False],
        'random_state': [42],
        'criterion': ['gini', 'entropy'],
        'n_jobs': [-1],
        'class_weight' : [None, 'balanced']
        }
        
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)

        rsh.fit(train_x_fp, train_y)

        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)                      
        classifier.fit(train_x_fp, train_y)
        
        #Threshold Balancing
        cross_val_prob_fp = cross_val_predict(rsh.best_estimator_, train_x_fp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_fp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_fp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_fp))


        fp_proba = classifier.predict_proba(test_x_fp)[:,1]
        #fp_pred = classifier.predict(test_x_fp) 
        fp_pred  = [ 1 if x>best_thresh_fp  else 0 for x in fp_proba ] 
    
        #Cell Painting
        
        print("CP_Model_Running for assay")  
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)
        
        
        rsh.fit(train_x_cp, train_y)
        
        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_    
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)
        classifier.fit(train_x_cp, train_y)
        
        #Threshold Balancing
        cross_val_prob_cp = cross_val_predict(rsh.best_estimator_, train_x_cp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_cp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_cp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_cp))


        CP_proba = classifier.predict_proba(test_x_cp)[:,1]
        #CP_pred = classifier.predict(test_x_cp) 
        CP_pred  = [ 1 if x>best_thresh_cp  else 0 for x in CP_proba ] 
        
        fpr, tpr, _ = roc_curve(test_y, CP_proba)

        #combine results
        for StdInChI, fp_proba, fp_pred, CP_proba, CP_pred, true_val, ts, pc in zip(test.StdInChI, fp_proba, fp_pred, CP_proba, CP_pred, test_y, df_ts_highest.ts_nearest,  df_pc_highest.pc_highest):
            detail_list_batch_test.append([assay, StdInChI, fp_proba, fp_pred,best_thresh_fp, CP_proba, CP_pred, best_thresh_cp, true_val, ts, pc, "HeldOut"])
    
        return detail_list_batch_test

In [14]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split

master_detail_list=[]
for assay in tqdm(assay_list[:]):
    
    print("Assay ", assay )
    df_assay= pubchem_data[pd.notnull(pubchem_data[assay])]

    print("Total Compounds ", len(df_assay) )

    df_assay_inactive = df_assay[df_assay[assay] == 0]
    inactives_n=len(df_assay_inactive)
    print("Inactive Compounds ", inactives_n)

    df_assay_active = df_assay[df_assay[assay] == 1]
    actives_n=len(df_assay_active)
    print("Active Compounds ", actives_n)

    if(inactives_n > 3*actives_n):
        df_assay_inactive= df_assay_inactive.sample(3*actives_n, random_state=1)
        print("Inactive Compounds (after Undersamplimg) ", len(df_assay_inactive) )
        df_assay= df_assay_active.append(df_assay_inactive) 
        print("Total Compounds (after Undersamplimg) ", len(df_assay) )
        
    elif(actives_n > 3*inactives_n):
        df_assay_active= df_assay_active.sample(3*inactives_n, random_state=1)
        print("Active Compounds (after Undersamplimg) ", len(df_assay_active) )
        df_assay= df_assay_inactive.append(df_assay_active) 
        print("Total Compounds (after Undersamplimg) ", len(df_assay) )
        
    else:
        print("No Undersamplimg; Assay already in 1:3 ratio or less", len(df_assay) )
        
        
    df_assay= df_assay.reset_index(drop=True)
    
    df_assay_train, df_assay_test= train_test_split(df_assay, stratify=df_assay[[assay]], test_size=0.20, random_state=42)
    
    print("Perform nested cross validation on training set")
    detail_list_train =  my_function_nested_rhs(assay, 42)
   
    
    print("Predictions on held out set")
    detail_list_test = my_function_heldout_rhs(assay, 42)

    
    master_detail_list.append(detail_list_train) 
    master_detail_list.append(detail_list_test) 
    
    

  0%|                                                    | 0/92 [00:00<?, ?it/s]

Assay  588458
Total Compounds  8083
Inactive Compounds  7971
Active Compounds  112
Inactive Compounds (after Undersamplimg)  336
Total Compounds (after Undersamplimg)  448
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 11, 'min_samples_split': 13, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.278423
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.306456
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 13, 'n_estimators': 400, 'n_jobs': -1, 'oo

  1%|▍                                        | 1/92 [02:15<3:25:03, 135.21s/it]

Best Threshold=0.318929
Assay  588334
Total Compounds  6665
Inactive Compounds  6533
Active Compounds  132
Inactive Compounds (after Undersamplimg)  396
Total Compounds (after Undersamplimg)  528
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 11, 'min_samples_split': 13, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.286914
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.291629
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 11, 'min_samples_split': 13, 'n_estimators

  2%|▉                                        | 2/92 [04:39<3:31:06, 140.73s/it]

...

 98%|███████████████████████████████████████▏| 90/92 [4:03:45<04:12, 126.15s/it]

Best Threshold=0.313440
Assay  1117304
Total Compounds  386
Inactive Compounds  260
Active Compounds  126
No Undersamplimg; Assay already in 1:3 ratio or less 386
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.339084
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 8, 'min_samples_split': 12, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.276130
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 11, 'min_samples_split': 13, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': 

 99%|███████████████████████████████████████▌| 91/92 [4:06:02<02:09, 129.50s/it]

Best Threshold=0.366563
Assay  1117305
Total Compounds  359
Inactive Compounds  247
Active Compounds  112
No Undersamplimg; Assay already in 1:3 ratio or less 359
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 11, 'min_samples_split': 13, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.363819
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 11, 'min_samples_split': 13, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.370679
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': Fa

100%|████████████████████████████████████████| 92/92 [4:08:16<00:00, 161.92s/it]


In [15]:
flatList_master_detail_list = [ item for elem in master_detail_list for item in elem]
list_of_lists_df = pd.DataFrame(flatList_master_detail_list,columns=["assay","StdInChI","fp_proba", "fp_pred", "fp_threshold", "CP_proba", 
                                                     "CP_pred","CP_threshold","true", "ts", "pc", "Data"])
list_of_lists_df["MFP_Correct"] = list_of_lists_df["fp_pred"]==list_of_lists_df["true"]
list_of_lists_df["CP_Correct"] = list_of_lists_df["CP_pred"]==list_of_lists_df["true"]
list_of_lists_df

,assay,StdInChI,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.215985,0,0.278423,0.291141,0,0.306456,0.0,0.418605,0.430046,Training,True,True
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.240910,0,0.278423,0.253367,0,0.306456,0.0,0.409091,0.680534,Training,True,True
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.232871,0,0.278423,0.154150,0,0.306456,0.0,0.447368,0.701574,Training,True,True
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.142133,0,0.278423,0.359792,1,0.306456,1.0,0.341463,0.453313,Training,False,True
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.253998,0,0.278423,0.202499,0,0.306456,0.0,0.388889,0.646760,Training,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50467,1117305,InChI=1S/C12H9F3N2O2/c1-7-10(6-16-19-7)11(18)1...,0.340631,0,0.357222,0.357490,1,0.330242,0.0,0.296296,0.479437,HeldOut,True,False
50468,1117305,InChI=1S/C12H10N2/c1-8-12-10(6-7-13-8)9-4-2-3-...,0.242578,0,0.357222,0.244216,0,0.330242,0.0,0.363636,0.619949,HeldOut,True,True
50469,1117305,InChI=1S/C8H4Cl2N2O2/c9-3-1-5-6(2-4(3)10)12-8(...,0.403774,1,0.357222,0.352388,1,0.330242,1.0,0.333333,0.541732,HeldOut,True,True
50470,1117305,InChI=1S/C15H12O5/c16-9-3-1-8(2-4-9)13-7-12(19...,0.478660,1,0.357222,0.243638,0,0.330242,0.0,0.444444,0.506977,HeldOut,False,True


In [16]:
#Probability balancing to thresholds

new_results= pd.DataFrame()
new_new_results = pd.DataFrame()

for t in list(list_of_lists_df["fp_threshold"].unique()):
        
        detail_list_fold=list_of_lists_df[list_of_lists_df["fp_threshold"]==t]
        print(len(detail_list_fold["fp_proba"].values), "Compounds in Fold with Threshold: ", np.round(t, 3))
        #print(detail_list_fold["fp_proba"].values)
        k_list=[]
        for p in detail_list_fold["fp_proba"].values:
            if p<=t:
                #print(p)
                k=p*0.5/t

            else:
                #print(p)
                k=1-((1-p)*0.5)/(1-t)

            k_list.append(k)
        #print(k_list)
        detail_list_fold["fp_proba_scaled"]=k_list
        new_results=pd.concat([new_results, detail_list_fold])
       
for t in list(new_results["CP_threshold"].unique()):
        
        detail_list_fold=new_results[new_results["CP_threshold"]==t]
        print(len(detail_list_fold["fp_proba"].values), "Compounds in Fold with Threshold: ", np.round(t, 3))
        #print(detail_list_fold["fp_proba"].values)
        k_list=[]
        for p in detail_list_fold["CP_proba"].values:
            if p<=t:
                #print(p)
                k=p*0.5/t

            else:
                #print(p)
                k=1-((1-p)*0.5)/(1-t)

            k_list.append(k)
        #print(k_list)
        detail_list_fold["CP_proba_scaled"]=k_list    
        
        new_new_results=pd.concat([new_new_results, detail_list_fold])

new_new_results

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


72 Compounds in Fold with Threshold:  0.278
72 Compounds in Fold with Threshold:  0.268
72 Compounds in Fold with Threshold:  0.302
71 Compounds in Fold with Threshold:  0.236
71 Compounds in Fold with Threshold:  0.304
90 Compounds in Fold with Threshold:  0.289
85 Compounds in Fold with Threshold:  0.287
85 Compounds in Fold with Threshold:  0.203
84 Compounds in Fold with Threshold:  0.251
84 Compounds in Fold with Threshold:  0.247
84 Compounds in Fold with Threshold:  0.26
106 Compounds in Fold with Threshold:  0.285
123 Compounds in Fold with Threshold:  0.317
122 Compounds in Fold with Threshold:  0.275
122 Compounds in Fold with Threshold:  0.25
122 Compounds in Fold with Threshold:  0.249
122 Compounds in Fold with Threshold:  0.264
153 Compounds in Fold with Threshold:  0.26
91 Compounds in Fold with Threshold:  0.224
91 Compounds in Fold with Threshold:  0.237
91 Compounds in Fold with Threshold:  0.236
91 Compounds in Fold with Threshold:  0.247
90 Compounds in Fold with Th

95 Compounds in Fold with Threshold:  0.24
80 Compounds in Fold with Threshold:  0.292
80 Compounds in Fold with Threshold:  0.269
80 Compounds in Fold with Threshold:  0.264
80 Compounds in Fold with Threshold:  0.269
80 Compounds in Fold with Threshold:  0.3
100 Compounds in Fold with Threshold:  0.272
75 Compounds in Fold with Threshold:  0.33
75 Compounds in Fold with Threshold:  0.297
75 Compounds in Fold with Threshold:  0.223
75 Compounds in Fold with Threshold:  0.296
74 Compounds in Fold with Threshold:  0.303
94 Compounds in Fold with Threshold:  0.273
20 Compounds in Fold with Threshold:  0.301
19 Compounds in Fold with Threshold:  0.288
19 Compounds in Fold with Threshold:  0.28
19 Compounds in Fold with Threshold:  0.343
19 Compounds in Fold with Threshold:  0.304
25 Compounds in Fold with Threshold:  0.444
21 Compounds in Fold with Threshold:  1.494
21 Compounds in Fold with Threshold:  0.417
21 Compounds in Fold with Threshold:  1.435
20 Compounds in Fold with Threshold:

61 Compounds in Fold with Threshold:  0.524
60 Compounds in Fold with Threshold:  0.554
76 Compounds in Fold with Threshold:  0.541
16 Compounds in Fold with Threshold:  0.265
16 Compounds in Fold with Threshold:  0.257
16 Compounds in Fold with Threshold:  0.285
16 Compounds in Fold with Threshold:  0.275
16 Compounds in Fold with Threshold:  0.227
20 Compounds in Fold with Threshold:  0.287
17 Compounds in Fold with Threshold:  0.521
17 Compounds in Fold with Threshold:  0.479
17 Compounds in Fold with Threshold:  0.473
17 Compounds in Fold with Threshold:  0.475
17 Compounds in Fold with Threshold:  0.495
22 Compounds in Fold with Threshold:  0.504
17 Compounds in Fold with Threshold:  0.267
17 Compounds in Fold with Threshold:  0.281
17 Compounds in Fold with Threshold:  0.236
16 Compounds in Fold with Threshold:  0.247
16 Compounds in Fold with Threshold:  0.295
21 Compounds in Fold with Threshold:  0.335
114 Compounds in Fold with Threshold:  0.697
114 Compounds in Fold with Thre

75 Compounds in Fold with Threshold:  0.26
74 Compounds in Fold with Threshold:  0.327
74 Compounds in Fold with Threshold:  0.3
74 Compounds in Fold with Threshold:  0.256
74 Compounds in Fold with Threshold:  0.264
93 Compounds in Fold with Threshold:  0.283
80 Compounds in Fold with Threshold:  0.266
80 Compounds in Fold with Threshold:  0.257
80 Compounds in Fold with Threshold:  0.29
80 Compounds in Fold with Threshold:  0.423
80 Compounds in Fold with Threshold:  0.246
100 Compounds in Fold with Threshold:  0.183
73 Compounds in Fold with Threshold:  0.317
73 Compounds in Fold with Threshold:  0.315
73 Compounds in Fold with Threshold:  0.392
73 Compounds in Fold with Threshold:  0.292
72 Compounds in Fold with Threshold:  0.319
92 Compounds in Fold with Threshold:  0.274
79 Compounds in Fold with Threshold:  0.245
79 Compounds in Fold with Threshold:  0.304
79 Compounds in Fold with Threshold:  0.252
78 Compounds in Fold with Threshold:  0.228
78 Compounds in Fold with Threshold

122 Compounds in Fold with Threshold:  0.597
122 Compounds in Fold with Threshold:  0.373
121 Compounds in Fold with Threshold:  0.542
153 Compounds in Fold with Threshold:  0.466
133 Compounds in Fold with Threshold:  0.429
133 Compounds in Fold with Threshold:  0.425
133 Compounds in Fold with Threshold:  0.402
133 Compounds in Fold with Threshold:  0.406
132 Compounds in Fold with Threshold:  0.45
166 Compounds in Fold with Threshold:  0.472
124 Compounds in Fold with Threshold:  0.428
124 Compounds in Fold with Threshold:  0.328
123 Compounds in Fold with Threshold:  0.324
123 Compounds in Fold with Threshold:  0.384
123 Compounds in Fold with Threshold:  0.406
155 Compounds in Fold with Threshold:  0.376
127 Compounds in Fold with Threshold:  0.312
127 Compounds in Fold with Threshold:  0.349
126 Compounds in Fold with Threshold:  0.306
126 Compounds in Fold with Threshold:  0.424
126 Compounds in Fold with Threshold:  0.278
158 Compounds in Fold with Threshold:  0.311
26 Compound

20 Compounds in Fold with Threshold:  0.585
26 Compounds in Fold with Threshold:  0.792
48 Compounds in Fold with Threshold:  0.622
48 Compounds in Fold with Threshold:  0.758
48 Compounds in Fold with Threshold:  0.638
48 Compounds in Fold with Threshold:  0.646
48 Compounds in Fold with Threshold:  0.698
61 Compounds in Fold with Threshold:  0.694
141 Compounds in Fold with Threshold:  0.169
141 Compounds in Fold with Threshold:  0.188
141 Compounds in Fold with Threshold:  0.262
141 Compounds in Fold with Threshold:  0.267
140 Compounds in Fold with Threshold:  0.111
176 Compounds in Fold with Threshold:  0.313
62 Compounds in Fold with Threshold:  0.276
62 Compounds in Fold with Threshold:  0.38
62 Compounds in Fold with Threshold:  0.33
61 Compounds in Fold with Threshold:  0.332
61 Compounds in Fold with Threshold:  0.333
78 Compounds in Fold with Threshold:  0.367
58 Compounds in Fold with Threshold:  0.371
58 Compounds in Fold with Threshold:  0.252
57 Compounds in Fold with Th

,assay,StdInChI,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.215985,0,0.278423,0.291141,0,0.306456,0.0,0.418605,0.430046,Training,True,True,0.387873,0.475014
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.240910,0,0.278423,0.253367,0,0.306456,0.0,0.409091,0.680534,Training,True,True,0.432633,0.413382
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.232871,0,0.278423,0.154150,0,0.306456,0.0,0.447368,0.701574,Training,True,True,0.418196,0.251504
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.142133,0,0.278423,0.359792,1,0.306456,1.0,0.341463,0.453313,Training,False,True,0.255247,0.538452
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.253998,0,0.278423,0.202499,0,0.306456,0.0,0.388889,0.646760,Training,True,True,0.456137,0.330388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50467,1117305,InChI=1S/C12H9F3N2O2/c1-7-10(6-16-19-7)11(18)1...,0.340631,0,0.357222,0.357490,1,0.330242,0.0,0.296296,0.479437,HeldOut,True,False,0.476778,0.520342
50468,1117305,InChI=1S/C12H10N2/c1-8-12-10(6-7-13-8)9-4-2-3-...,0.242578,0,0.357222,0.244216,0,0.330242,0.0,0.363636,0.619949,HeldOut,True,True,0.339534,0.369752
50469,1117305,InChI=1S/C8H4Cl2N2O2/c9-3-1-5-6(2-4(3)10)12-8(...,0.403774,1,0.357222,0.352388,1,0.330242,1.0,0.333333,0.541732,HeldOut,True,True,0.536212,0.516533
50470,1117305,InChI=1S/C15H12O5/c16-9-3-1-8(2-4-9)13-7-12(19...,0.478660,1,0.357222,0.243638,0,0.330242,0.0,0.444444,0.506977,HeldOut,False,True,0.594464,0.368877


In [17]:
new_new_results.to_csv("Predictions_train_heldout_scaled_prob_all_assays.csv", index=False)